In [1]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [2]:
!pip install python-whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103246 sha256=95601ad0193d20ec0da2269f6a29d4f3c37552b0e113d6d356e18b8b35118c0a
  Stored in directory: /root/.cache/pip/wheels/10/f1/87/145023b9a206e2e948be6480c61ef3fd3dbb81ef11b6977782
Successfully built python-whois


In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
import ssl, socket
from datetime import datetime
import whois
import json

In [4]:
from tensorflow.keras.preprocessing import sequence
max_review_length = 60 #maximum charater lenght in domain name

In [5]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('modelNew91_59.h5')

In [6]:
charMap={}
#All characters are encoded into a number during training
with open('charencode.json') as json_file:
    charMap = json.load(json_file)

In [ ]:
charMap

In [9]:
# SSL issuers are numbered during training
sslMap={}
with open("data/sslIssuers.json") as outfile:
   sslMap = json.load(outfile)

In [10]:
sslMap

{'Google Trust Services LLC': 0,
 "Let's Encrypt": 1,
 'cPanel, Inc.': 2,
 'Isimtescil Bilisim A.S.': 3,
 'Cloudflare, Inc.': 4,
 'GoDaddy.com, Inc.': 5,
 'Sectigo Limited': 6,
 'NaN': 7,
 'GlobalSign nv-sa': 8,
 'DigiCert Inc': 9,
 'Gandi': 10,
 'ZeroSSL': 11,
 'Unizeto Technologies S.A.': 12,
 'Entrust, Inc.': 13,
 'COMODO CA Limited': 14,
 'SSL Corporation': 15,
 'DigiCert, Inc.': 16,
 'E-TUGRA EBG BILISIM TEKNOLOJILERI VE HIZMETLERI ANONIM SIRKETI': 17,
 'Amazon': 18,
 'Starfield Technologies, Inc.': 19,
 'GoGetSSL': 20,
 'Corporation Service Company': 21,
 'Trust Provider B.V.': 22,
 'CentralNic Luxembourg Sàrl': 23,
 'TrustAsia Technologies, Inc.': 24}

In [11]:
#is used to list of the encoded charaters for a given web page
charDict={}
def asciize(strIn):
    global charMap
    charList=[]
    #print(strIn)
    for ch in str(strIn):
      charList.append(charMap[ch])
    return charList

In [12]:
#Extracting domain name from given web site
def getDomainName(url):
    dom=url
    if "." in url:
            urlList=url.split(".")
            ln=len(urlList)
            if ln==2:
                dom=urlList[0]
                return dom
            elif ln==3:
                dom=urlList[0]
                return dom
            elif ln==4:
                dom=urlList[1]
                return dom
            elif not dom.isnumeric():
                return dom

    else:
        return dom

In [13]:
# This function is used to predict phishing score of given domain name, prediction is made using trained deep learning model
# score near to 1 is dangerous phishing web site, near to 0 is legal website
def predictUrl(url):
  url=getDomainName(url)
  arr = [np.array(asciize(url))]
  #print(arr)
  my_test = sequence.pad_sequences(arr, maxlen=60, padding='post',truncating='post')
  testScore=model.predict(my_test)
  return testScore[0][0]

In [18]:
#Please go to https://www.usom.gov.tr/adres and get a newly detected website and test it.
predictUrl("dijitalbasvurugiris-denizacik.com")

1/1 [==============================] - 0s 18ms/step


0.99995804

In [19]:
#This function will be used to get ssl certification details of a given web page
# For phase2 in machine learning model
def getSslCertDetails(hostname):
    try:
      ctx = ssl.create_default_context()
      with ctx.wrap_socket(socket.socket(), server_hostname=hostname) as s:
          s.connect((hostname, 443))
          cert = s.getpeercert()

      dateTimeStr="%b %d %H:%M:%S %Y %Z"
      certStart=cert["notBefore"]
      certEnd=cert["notAfter"]
      certEnd=datetime.strptime(certEnd,dateTimeStr).date()
      certStart=datetime.strptime(certStart,dateTimeStr).date()
      current_dateTime = datetime.now().date()
      willEnd=(certEnd - current_dateTime).days
      startedBefore=(current_dateTime-certStart).days
      subject = dict(x[0] for x in cert['subject'])
      issued_to = subject['commonName']
      issuer = dict(x[0] for x in cert['issuer'])
      issued_by = issuer['organizationName']
      return willEnd,startedBefore,issued_by
    except:
        return None

In [20]:
#This function will be used to get domain name details to be used in phase2
# For phase2 in machine learning model
def whoisCheck(url):
    try:
      whoIs=whois.whois(url)
      urlDict={}
      if whoIs['domain_name'] is not None:
        current_dateTime = datetime.now()
        dateTimeStr="%Y-%m-%d %H:%M:%S"
        current_dateTime = datetime.now()
        #print(whoIs['creation_date'])
        if type(whoIs['domain_name']) == list:
          whoIs['domain_name']=whoIs['domain_name'][1]
        urlDict['domain_name']=whoIs['domain_name']
        if isinstance(whoIs['creation_date'], datetime):
          urlDict['domain_start']=(current_dateTime-whoIs['creation_date']).days
          urlDict['domain_end']=(whoIs['expiration_date']-current_dateTime).days
        elif type(whoIs['creation_date']) == list:
          urlDict['domain_start']=(current_dateTime-whoIs['creation_date'][0]).days
          urlDict['domain_end']=(whoIs['expiration_date'][0]-current_dateTime).days
        else:
          urlDict['domain_start']=datetime.strptime(whoIs['creation_date'],dateTimeStr).date()
          urlDict['domain_end']=datetime.strptime(whoIs['expiration_date'],dateTimeStr).date()
          urlDict['domain_start']=(current_dateTime-urlDict['domain_start']).days
          urlDict['domain_end']=(current_dateTime-urlDict['domain_end']).days
        if "city" in whoIs:
          urlDict['domain_city']=whoIs['city']
          urlDict['domain_state']=whoIs['state']
          urlDict['domain_country']=whoIs['country']
        if "domain_registrar" in whoIs:
          urlDict['domain_registrar']=whoIs['registrar']
        return urlDict['domain_start'],urlDict['domain_end']
    except:
        return None

In [21]:
def getPh2Data(url):
  ph1Score=predictUrl(url)
  myDict={}
  dmStart=-1
  dmEnd=-1
  myUrl=url
  try:
      dmStart,dmEnd=whoisCheck(url)
  except:
    print("Domain details can not be retreived")
    return None,ph1Score
  try:
      sslEnd,sslStart,sslIssuer=getSslCertDetails(url)
  except:
      sslStart=-1
      sslEnd=-1
      sslIssuer=None
  if "." in url:
      urlList=url.split(".")
      ln=len(urlList)
      if ln==2:
          dom=urlList[0]
      if ln==3:
          dom=urlList[0]
      if ln==4:
          dom=urlList[1]
      #if not dom.isnumeric():
      if sslIssuer is None or sslIssuer not in sslMap:
        sslNum=7
      else:
        sslNum=sslMap[sslIssuer]
      myDict = {'domainStart' : [dmStart], 'domainEnd' : [dmEnd],'sslStart':[sslStart],'sslEnd':[sslEnd],'dlPhishingScore':[ph1Score],'sslIssuerNum':[sslNum]}
  return myUrl,myDict


In [22]:
getPh2Data("market-ocak.com")

1/1 [==============================] - 0s 29ms/step


('market-ocak.com',
 {'domainStart': [0],
  'domainEnd': [365],
  'sslStart': [0],
  'sslEnd': [90],
  'dlPhishingScore': [0.19598332],
  'sslIssuerNum': [1]})

,domainStart,domainEnd,sslStart,sslEnd,dlPhishingScore,sslIssuerNum
0,9954,1001,323,74,0.078849,8


In [23]:
from sklearn import model_selection
import pickle
from sklearn.linear_model import LogisticRegression
# Support Vector Machines
from sklearn.svm import LinearSVC
# Decision Trees
from sklearn.tree import DecisionTreeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier



In [24]:
from sklearn.preprocessing import MinMaxScaler
from pickle import load

In [25]:
LR_model = pickle.load(open('data/Logistic Regression.sav', 'rb'))
RF_model = pickle.load(open('data/Random Forest.sav', 'rb'))
DT_model = pickle.load(open('data/Decision Trees.sav', 'rb'))
SVM_model = pickle.load(open('data/Support Vector Machines.sav', 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at y

In [26]:
loaded_scaler = load(open('scaler.pkl', 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
def isPhishing(testUrl):
  myUrl,myDict=getPh2Data(testUrl)
  if myUrl is None:
    print(f"It looks like this domain is not exist at the moment, but character level phishing score is:{myDict}")
    return None
  sampleDf=pd.DataFrame.from_dict(myDict)
  print(sampleDf)
  scaledSample=loaded_scaler.transform(sampleDf)
  lrPred=LR_model.predict(scaledSample)
  rfPred=RF_model.predict(scaledSample)
  dtPred=DT_model.predict(scaledSample)
  svmPred=SVM_model.predict(scaledSample)
  return {"url":testUrl,"lr":lrPred[0],"rf":rfPred[0],"dt":dtPred[0],"svm":svmPred[0]}

In [40]:
result=isPhishing("hasanustakebap.com")

1/1 [==============================] - 0s 20ms/step
   domainStart  domainEnd  sslStart  sslEnd  dlPhishingScore  sslIssuerNum
0         7304          0         8      82         0.044081             1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


In [41]:
print(f'phishing prediction with Logistic Regression for {result["url"]}...:{result["lr"]}')
print(f'phishing prediction with Random Forest for {result["url"]}...:{result["rf"]}')
print(f'phishing prediction with Decision Trees for {result["url"]}...:{result["dt"]}')
print(f'phishing prediction with SVM for {result["url"]}...:{result["svm"]}')

phishing prediction with Logistic Regression for hasanustakebap.com...:0
phishing prediction with Random Forest for hasanustakebap.com...:1
phishing prediction with Decision Trees for hasanustakebap.com...:1
phishing prediction with SVM for hasanustakebap.com...:1
